# LIDC-IDRI Lung Nodule Patch Dataset Creation

## Overview
This notebook extracts representative 2D patches from the LIDC-IDRI (Lung Image Database Consortium and Image Database Resource Initiative) dataset for machine learning applications in lung nodule analysis.

## Key Features
- **Multi-radiologist Consensus**: Combines annotations from multiple radiologists using configurable consensus thresholds
- **Intelligent Slice Selection**: Selects representative slices based on nodule area and spatial proximity
- **Standardized CT Windowing**: Applies lung window settings (-600 HU ± 750 HU) for consistent visualization
- **Flexible Output**: Generates both individual patch images and comprehensive metadata CSV files
- **Robust Fallback Strategies**: Handles edge cases with empty consensus masks

## Dataset Information
- **Source**: LIDC-IDRI public dataset
- 1018 scans from 1010 patients.
- Each scan includes multiple nodules annotated by up to 4 radiologists.
- **Annotations**: up to 4 radiologists per nodule with 9 semantic attributes
- **Output Format**: PNG images (nodule patches) + CSV metadata with annotation scores. Specific fields see documentation.

## Usage
- Specify patient ID for single-patient processing
- Perform full dataset extraction
- Modify parameters as needed (consensus level, area threshold, etc.)


---

In [1]:
import os
from typing import Dict, List, Tuple, Any
import numpy as np
import pandas as pd
import pylidc as pl
from pylidc.utils import consensus
from tqdm import tqdm

from PIL import Image

In [ ]:
# ===============================================================================
# 1) UTILITY FUNCTIONS
# ===============================================================================

# Annotation fields from LIDC-IDRI dataset (9 semantic attributes)
ANNOT_FIELDS = [
    'subtlety', 'internalStructure', 'calcification',
    'sphericity', 'margin', 'lobulation', 'spiculation',
    'texture', 'malignancy',
]

def _inplane_area_mm2(scan) -> float:
    """
    Calculate the area of each pixel in the axial plane (mm^2).
    
    Args:
        scan: pylidc Scan object
        
    Returns:
        float: Area per pixel in mm^2
    """
    ps = scan.pixel_spacing  # Pixel spacing from pylidc
    try:
        sx, sy = float(ps[0]), float(ps[1])
    except Exception:
        sx = sy = float(ps)
    return sx * sy  # mm^2/px

def _aggregate_ann_fields(anns, agg="round") -> Dict[str, int]:
    """
    Aggregate annotations from multiple radiologists for the same nodule.
    
    Annotation fields include: 'subtlety', 'internalStructure', 'calcification',
    'sphericity', 'margin', 'lobulation', 'spiculation', 'texture', 'malignancy'.
    
    Args:
        anns: List of pylidc Annotation objects from multiple radiologists
        agg: Aggregation method - "round" (mean then round) or "mean" (truncated mean)
        
    Returns:
        Dict[str, int]: Aggregated annotation scores
    """
    out = {}
    for f in ANNOT_FIELDS:
        vals = [getattr(a, f) for a in anns if getattr(a, f) is not None]
        if agg == "round":
            out[f] = int(np.rint(np.mean(vals))) if len(vals) else None
        elif agg == "mean":
            out[f] = int(np.mean(vals)) if len(vals) else None
    return out

def get_final_indices(kept, neigh, mode="intersect"):
    """
    Combine area-filtered slices with max-area neighboring slices.
    
    Args:
        kept: Slice indices that pass area threshold
        neigh: Neighboring slice indices around max-area slice
        mode: "union" (combine all) or "intersect" (only overlapping)
        
    Returns:
        np.ndarray: Final selected slice indices
    """
    if mode == "union":
        return np.unique(np.concatenate([kept, np.array(neigh, dtype=int)]))
    elif mode == "intersect":
        return np.intersect1d(kept, neigh)
    else:
        raise ValueError("mode must be 'union' or 'intersect'")

def window_hu(img_hu, wl=-600, ww=1500):
    """
    Apply CT windowing (lung window) to convert HU values to 0-255 grayscale.
    
    Standard lung window: WL=-600, WW=1500 (range: -1350 to 150 HU)
    
    Args:
        img_hu: CT image in Hounsfield Units
        wl: Window level (center)
        ww: Window width
        
    Returns:
        np.ndarray: 8-bit grayscale image (0-255)
    """
    lo, hi = wl - ww/2.0, wl + ww/2.0
    x = np.clip(img_hu, lo, hi)
    x = (x - lo) / (hi - lo)
    return (x * 255).astype(np.uint8)

def _select_repr_slices_from_cmask(
    cmask: np.ndarray,
    scan,
    min_area_mm2: float = 50.0,   # Minimum nodule area threshold (adjustable)
    drop_ends: bool = False,      # Whether to drop first/last slices
    n_neighbors: int = 2,         # Number of neighboring slices around max-area slice
    mode: str = "intersect"       # Intersection or union of filtering criteria
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Select representative slices from consensus volume mask based on:
    1. Optional removal of first/last slices (boundary artifacts)
    2. Area-based filtering (remove slices with area < min_area_mm2)
    3. Forced inclusion of max-area slice ± neighboring slices
    
    Args:
        cmask: 3D consensus mask (H, W, K)
        scan: pylidc Scan object for pixel spacing
        min_area_mm2: Minimum area threshold in mm^2
        drop_ends: Whether to exclude first/last slices
        n_neighbors: Number of slices above/below max-area slice to include
        mode: "intersect" or "union" for combining area and neighbor criteria
        
    Returns:
        Tuple[np.ndarray, np.ndarray]: (selected slice indices, corresponding areas)
    """
    assert cmask.ndim == 3, "cmask expected to be (H, W, K)"
    H, W, K = cmask.shape

    # Calculate area for each slice
    px_area = _inplane_area_mm2(scan)
    areas = np.array([(cmask[:, :, k].sum()) * px_area for k in range(K)])

    # Initialize slice indices
    idx = np.arange(K)
    if drop_ends and K >= 3:
        idx = idx[1:-1]  # Remove first and last slices

    # Filter by area threshold
    kept = idx[areas[idx] >= float(min_area_mm2)]
    if kept.size == 0:
        # Fallback: keep at least the max-area slice
        kept = np.array([int(np.argmax(areas))])

    # Always include max-area slice and its neighbors
    k0 = int(np.argmax(areas))
    neigh = [k0 + d for d in range(-n_neighbors, n_neighbors + 1) 
             if 0 <= k0 + d < K]  # Clamp to valid range
    
    # Combine filtering results
    kept_final = get_final_indices(kept, neigh, mode=mode)

    return kept_final, areas[kept_final]

# ===============================================================================
# 2) CORE FUNCTION: Extract patches for a single nodule
# ===============================================================================

def extract_patches_for_nodule(
    scan,
    anns: List[pl.Annotation],
    clevel: float = 0.5,
    pad: List[Tuple[int, int]] = [(25, 25), (25, 25), (0, 0)],
    drop_ends: bool = False,
    min_area_mm2: float = 50.0,
    n_neighbors: int = 2,
    mode: str = "intersect",
) -> Dict[str, Any]:
    """
    Extract representative 2D patches from a 3D nodule annotation.
    
    Process:
    1. Generate consensus mask from multiple radiologist annotations
    2. Apply fallback strategies if consensus is empty
    3. Select representative slices based on area and spatial criteria
    4. Extract 2D image/mask patches for each selected slice
    
    Args:
        scan: pylidc Scan object
        anns: List of Annotation objects from multiple radiologists
        clevel: Consensus level (0.5 = majority agreement)
        pad: Padding around nodule bounding box [(x_pad), (y_pad), (z_pad)]
        drop_ends: Whether to exclude first/last slices of the nodule
        min_area_mm2: Minimum area threshold for slice selection
        n_neighbors: Number of neighboring slices around max-area slice
        mode: Slice selection mode ("intersect" or "union")
        
    Returns:
        Dict containing:
        - 'bbox': Consensus bounding box
        - 'patches': List of patch dictionaries with img, mask, indices, area
        - 'ann_agg': Aggregated annotation scores from multiple radiologists
    """
    # Load full CT volume
    vol = scan.to_volume()  # Shape: (H, W, K) in HU values
    
    # Generate consensus mask (with fallback strategies)
    cmask, cbbox, masks = consensus(anns, clevel=clevel, pad=pad)
    
    if cmask.sum() == 0:
        # Fallback 1: Lower consensus threshold
        for cl in (0.25, 0.1):
            cmask_try, _, _ = consensus(anns, clevel=cl, pad=pad)
            if cmask_try.sum() > 0:
                cmask = cmask_try
                break
        
        # Fallback 2: Use union of all individual masks
        if cmask.sum() == 0 and len(masks) > 0:
            cmask = np.any(np.stack(masks, axis=0), axis=0).astype(bool)

    # Extract subvolume using consensus bounding box
    subvol = vol[cbbox]
    
    # Select representative slices
    kept_k, kept_areas = _select_repr_slices_from_cmask(
        cmask, scan, min_area_mm2=min_area_mm2, n_neighbors=n_neighbors, mode=mode,
        drop_ends=drop_ends
    )

    # Generate 2D patches for selected slices
    patches = []
    for k, k_area in zip(kept_k, kept_areas):
        img2d = subvol[:, :, int(k)]  # 2D CT slice
        m2d = cmask[:, :, int(k)].astype(bool)  # 2D binary mask
        
        patches.append({
            "img": img2d,                         # 2D HU patch
            "mask": m2d,                          # 2D boolean mask
            "k_local": int(k),                    # Index within bounding box
            "k_global": int(cbbox[2].start + k),  # Index in original volume
            "area": k_area,                       # Slice area in mm^2
        })

    # Aggregate annotations from multiple radiologists
    ann_agg = _aggregate_ann_fields(anns)

    return {"bbox": cbbox, "patches": patches, "ann_agg": ann_agg}

# ===============================================================================
# 3) TOP-LEVEL: Patient-wise batch processing
# ===============================================================================

def extract_patient_level_patches(
    patient_id: str = None,
    clevel: float = 0.5,
    pad: List[Tuple[int, int]] = [(25, 25), (25, 25), (0, 0)],
    drop_ends: bool = False,
    min_area_mm2: float = 50.0,
    n_neighbors: int = 2,
    mode: str = "intersect"
):
    """
    Extract patches for all nodules in specified patient(s).
    
    Processing pipeline:
    1. Query LIDC-IDRI database for patient scan(s)
    2. Cluster annotations by spatial proximity (nodule grouping)
    3. Extract representative patches for each nodule
    4. Yield structured results for downstream processing
    
    Args:
        patient_id: Specific patient ID (e.g., "LIDC-IDRI-0001") or None for all
        clevel: Consensus level for multi-radiologist agreement
        pad: Bounding box padding
        drop_ends: Whether to exclude first/last slices of each nodule
        min_area_mm2: Minimum area threshold for slice filtering
        n_neighbors: Number of neighboring slices around max-area slice
        mode: Slice selection mode ("intersect" or "union")
        
    Yields:
        Dict containing patient metadata and extracted patches
    """
    q = pl.query(pl.Scan)
    if patient_id:
        q = q.filter(pl.Scan.patient_id == patient_id)
    
    for scan in q:  # Each scan = one patient CT examination
        nodule_groups = scan.cluster_annotations()  # Group annotations by spatial proximity
        
        for n_idx, anns in enumerate(nodule_groups):  # Each nodule cluster
            res = extract_patches_for_nodule(
                scan, anns,
                clevel=clevel, pad=pad,
                min_area_mm2=min_area_mm2, n_neighbors=n_neighbors,
                mode=mode, drop_ends=drop_ends
            )
            
            yield {
                "patient_id": scan.patient_id,
                "scan_id": scan.id,
                "nodule_index": int(n_idx),
                "bbox": res["bbox"],
                "ann_summary": res["ann_agg"],     # Aggregated annotation scores
                "patches": res["patches"],         # Representative slice patches
            }

def save_patches_and_metadata(
    out_dir: str,
    patient_id: str,
    metadata_csv: str,
    clevel: float = 0.5,
    pad: List[Tuple[int, int]] = [(25, 25), (25, 25), (0, 0)],
    drop_ends: bool = False,
    min_area_mm2: float = 50.0,
    n_neighbors: int = 2,
    mode: str = "intersect",
):
    """
    Extract patches and save as images with metadata CSV.
    
    Output structure:
    - Individual PNG files for each patch (image and mask)
    - CSV metadata file with patch information and annotation scores
    
    Args:
        out_dir: Output directory for patch images
        patient_id: Target patient ID (LIDC-IDRI-xxxx format)
        metadata_csv: Path for output CSV metadata file
        clevel: Consensus level for multi-radiologist agreement
        pad: Bounding box padding
        drop_ends: Whether to exclude first/last slices of each nodule
        min_area_mm2: Minimum area threshold (mm^2)
        n_neighbors: Number of neighboring slices around max-area slice
        mode: Slice selection mode ("intersect" or "union")
        
    Returns:
        pd.DataFrame: Generated metadata with patch information
    """
    os.makedirs(out_dir, exist_ok=True)
    records = []
    
    for item in extract_patient_level_patches(
        patient_id=patient_id,
        clevel=clevel,
        pad=pad,
        min_area_mm2=min_area_mm2,
        n_neighbors=n_neighbors,
        mode=mode,
        drop_ends=drop_ends
    ):
        sid = item["scan_id"]
        pid = item["patient_id"]
        nid = item["nodule_index"]

        for p in item["patches"]:
            k_global = p["k_global"]
            img = p["img"]
            mask = p["mask"]

            # Generate file paths
            img_path = os.path.join(out_dir, f"{pid}_n{nid}_k{k_global}_img.png")
            mask_path = os.path.join(out_dir, f"{pid}_n{nid}_k{k_global}_mask.png")

            # Apply CT windowing for standardized visualization
            img_windowed = window_hu(img)  # Lung window: filters noise and artifacts
            
            # Save images (avoiding per-image normalization for consistency)
            Image.fromarray(img_windowed).save(img_path)
            Image.fromarray((mask.astype(np.uint8) * 255)).save(mask_path)

            # bbox information
            nodule_bbox = item["bbox"]
            nodule_bbox = (
                nodule_bbox[0].start, nodule_bbox[1].start,
                nodule_bbox[0].stop, nodule_bbox[1].stop
            )  # (x_min, y_min, x_max, y_max)

            # Build metadata record
            record = {
                "scan_id": sid,
                "patient_id": pid,
                "nodule_index": nid,
                "k_global": k_global,
                "img_path": img_path,
                "mask_path": mask_path,
                "area_mm2": p["area"],
                "nodule_bbox_xmin": nodule_bbox[0],
                "nodule_bbox_ymin": nodule_bbox[1],
                "nodule_bbox_xmax": nodule_bbox[2],
                "nodule_bbox_ymax": nodule_bbox[3],
            }
            # Add aggregated annotation scores
            record.update({f"ann_{k}": v for k, v in item["ann_summary"].items()})

            records.append(record)

    # Save metadata to CSV
    df = pd.DataFrame(records)
    df.to_csv(metadata_csv, index=False)
    print(f"Metadata saved to {metadata_csv}, total patches: {len(df)}")

    return df

In [28]:
# ===============================================================================
# EXAMPLE 1: Process a single patient (LIDC-IDRI-0078)
# ===============================================================================

out_dir = "./lidc_patches_0078"
os.makedirs(out_dir, exist_ok=True)
target_pid = "LIDC-IDRI-0078"  # Example patient ID

save_patches_and_metadata(
    out_dir=out_dir,
    metadata_csv=os.path.join(out_dir, "patches_metadata.csv"),
    clevel=0.5,                    # 50% consensus threshold
    pad=[(25, 25), (25, 25), (0, 0)],  # 25-pixel padding in x,y; no z-padding
    min_area_mm2=50.0,             # Minimum nodule area: 50 mm^2
    n_neighbors=2,                 # Include ±2 slices around max-area slice
    mode="intersect",              # Use intersection of area and neighbor criteria
    patient_id=target_pid,
    drop_ends=False                 # Exclude first/last slices of each nodule
)

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Metadata saved to ./lidc_patches_0078\patches_metadata.csv, total patches: 16


,scan_id,patient_id,nodule_index,k_global,img_path,mask_path,area_mm2,nodule_bbox_xmin,nodule_bbox_ymin,nodule_bbox_xmax,nodule_bbox_ymax,ann_subtlety,ann_internalStructure,ann_calcification,ann_sphericity,ann_margin,ann_lobulation,ann_spiculation,ann_texture,ann_malignancy
0,1,LIDC-IDRI-0078,0,24,./lidc_patches_0078\LIDC-IDRI-0078_n0_k24_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n0_k24_mask...,156.3250,269,281,353,375,4,1,6,4,3,2,2,5,4
1,1,LIDC-IDRI-0078,0,25,./lidc_patches_0078\LIDC-IDRI-0078_n0_k25_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n0_k25_mask...,184.2100,269,281,353,375,4,1,6,4,3,2,2,5,4
2,1,LIDC-IDRI-0078,0,26,./lidc_patches_0078\LIDC-IDRI-0078_n0_k26_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n0_k26_mask...,191.3925,269,281,353,375,4,1,6,4,3,2,2,5,4
3,1,LIDC-IDRI-0078,0,27,./lidc_patches_0078\LIDC-IDRI-0078_n0_k27_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n0_k27_mask...,147.8750,269,281,353,375,4,1,6,4,3,2,2,5,4
4,1,LIDC-IDRI-0078,0,28,./lidc_patches_0078\LIDC-IDRI-0078_n0_k28_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n0_k28_mask...,81.1200,269,281,353,375,4,1,6,4,3,2,2,5,4
5,1,LIDC-IDRI-0078,1,44,./lidc_patches_0078\LIDC-IDRI-0078_n1_k44_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n1_k44_mask...,87.8800,121,324,211,401,5,1,6,4,3,3,2,4,4
6,1,LIDC-IDRI-0078,1,45,./lidc_patches_0078\LIDC-IDRI-0078_n1_k45_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n1_k45_mask...,182.0975,121,324,211,401,5,1,6,4,3,3,2,4,4
7,1,LIDC-IDRI-0078,1,46,./lidc_patches_0078\LIDC-IDRI-0078_n1_k46_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n1_k46_mask...,183.3650,121,324,211,401,5,1,6,4,3,3,2,4,4
8,1,LIDC-IDRI-0078,1,47,./lidc_patches_0078\LIDC-IDRI-0078_n1_k47_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n1_k47_mask...,148.2975,121,324,211,401,5,1,6,4,3,3,2,4,4
9,1,LIDC-IDRI-0078,1,48,./lidc_patches_0078\LIDC-IDRI-0078_n1_k48_img.png,./lidc_patches_0078\LIDC-IDRI-0078_n1_k48_mask...,125.9050,121,324,211,401,5,1,6,4,3,3,2,4,4


In [29]:
# ===============================================================================
# EXAMPLE 2: Process another patient (LIDC-IDRI-0151)
# ===============================================================================

out_dir = "./lidc_patches_0151"
os.makedirs(out_dir, exist_ok=True)
target_pid = "LIDC-IDRI-0151"  # Another example patient

save_patches_and_metadata(
    out_dir=out_dir,
    metadata_csv=os.path.join(out_dir, "patches_metadata.csv"),
    clevel=0.5,
    pad=[(25, 25), (25, 25), (0, 0)],
    min_area_mm2=50.0,
    n_neighbors=2,
    mode="intersect",
    patient_id=target_pid,
    drop_ends=False
)

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Metadata saved to ./lidc_patches_0151\patches_metadata.csv, total patches: 3


,scan_id,patient_id,nodule_index,k_global,img_path,mask_path,area_mm2,nodule_bbox_xmin,nodule_bbox_ymin,nodule_bbox_xmax,nodule_bbox_ymax,ann_subtlety,ann_internalStructure,ann_calcification,ann_sphericity,ann_margin,ann_lobulation,ann_spiculation,ann_texture,ann_malignancy
0,10,LIDC-IDRI-0151,0,74,./lidc_patches_0151\LIDC-IDRI-0151_n0_k74_img.png,./lidc_patches_0151\LIDC-IDRI-0151_n0_k74_mask...,50.777985,283,381,350,452,5,1,6,4,4,3,3,5,4
1,153,LIDC-IDRI-0151,0,71,./lidc_patches_0151\LIDC-IDRI-0151_n0_k71_img.png,./lidc_patches_0151\LIDC-IDRI-0151_n0_k71_mask...,90.560329,265,370,339,446,5,1,6,3,3,3,2,4,4
2,153,LIDC-IDRI-0151,0,72,./lidc_patches_0151\LIDC-IDRI-0151_n0_k72_img.png,./lidc_patches_0151\LIDC-IDRI-0151_n0_k72_mask...,96.572633,265,370,339,446,5,1,6,3,3,3,2,4,4


In [30]:
# ===============================================================================
# SCAN METADATA: Generate overview table for all patients in database
# ===============================================================================

# Query all scans in LIDC-IDRI database
scans_all = pl.query(pl.Scan)
scans_all = scans_all.all()

# Create metadata DataFrame with scan information
scan_metainfo = pd.DataFrame(scans_all, columns=['Scan_obj'])
scan_metainfo['id'] = scan_metainfo['Scan_obj'].apply(lambda x: x.id)
scan_metainfo['patient_id'] = scan_metainfo['Scan_obj'].apply(lambda x: x.patient_id)
scan_metainfo['num_annotations'] = scan_metainfo['Scan_obj'].apply(lambda x: len(x.annotations))

scan_metainfo

,Scan_obj,id,patient_id,num_annotations
0,"Scan(id=1,patient_id=LIDC-IDRI-0078)",1,LIDC-IDRI-0078,13
1,"Scan(id=2,patient_id=LIDC-IDRI-0069)",2,LIDC-IDRI-0069,9
2,"Scan(id=3,patient_id=LIDC-IDRI-0079)",3,LIDC-IDRI-0079,4
3,"Scan(id=4,patient_id=LIDC-IDRI-0101)",4,LIDC-IDRI-0101,2
4,"Scan(id=5,patient_id=LIDC-IDRI-0110)",5,LIDC-IDRI-0110,6
...,...,...,...,...
1013,"Scan(id=1014,patient_id=LIDC-IDRI-0641)",1014,LIDC-IDRI-0641,21
1014,"Scan(id=1015,patient_id=LIDC-IDRI-0640)",1015,LIDC-IDRI-0640,7
1015,"Scan(id=1016,patient_id=LIDC-IDRI-0639)",1016,LIDC-IDRI-0639,5
1016,"Scan(id=1017,patient_id=LIDC-IDRI-0638)",1017,LIDC-IDRI-0638,3


In [31]:
# ===============================================================================
# BATCH PROCESSING: Extract patches for all patients in database
# ===============================================================================

metadata_df_all = []
directory = "./lidc_patches_all"
os.makedirs(directory, exist_ok=True)

# Get list of all unique patient IDs
# patient_list = scan_metainfo.patient_id.unique()
patient_list = ["LIDC-IDRI-0078", "LIDC-IDRI-0151", "LIDC-IDRI-0115", "LIDC-IDRI-0054"]  # Uncomment for testing with subset

# Process each patient sequentially with progress bar
for patient in tqdm(patient_list, desc="Processing patients"):
    out_dir = os.path.join(directory, patient)
    os.makedirs(out_dir, exist_ok=True)
    target_pid = patient

    # Extract patches for current patient
    metadata_df_patient = save_patches_and_metadata(
        out_dir=out_dir,
        metadata_csv=os.path.join(out_dir, "patches_metadata.csv"),
        clevel=0.5,
        pad=[(25, 25), (25, 25), (0, 0)],
        drop_ends=False,
        min_area_mm2=50.0,
        n_neighbors=2,
        mode="intersect",
        patient_id=target_pid
    )

    # Accumulate metadata from all patients
    metadata_df_all.append(metadata_df_patient)

# Combine all patient metadata into single DataFrame
metadata_df = pd.concat(metadata_df_all, ignore_index=True)
metadata_df.to_csv(os.path.join(directory, "all_patches_metadata.csv"), index=False)

print(f"Batch processing complete!")
print(f"Output directory: {directory}")
print(f"Total patches extracted: {len(metadata_df)}")
print(f"Patients processed: {len(patient_list)}")

Processing patients:   0%|          | 0/4 [00:00<?, ?it/s]

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.


Processing patients:  25%|██▌       | 1/4 [00:03<00:10,  3.56s/it]

Metadata saved to ./lidc_patches_all\LIDC-IDRI-0078\patches_metadata.csv, total patches: 16
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.


Processing patients:  50%|█████     | 2/4 [00:05<00:05,  2.70s/it]

Metadata saved to ./lidc_patches_all\LIDC-IDRI-0151\patches_metadata.csv, total patches: 3
Loading dicom files ... This may take a moment.


Processing patients:  75%|███████▌  | 3/4 [00:06<00:02,  2.01s/it]

Metadata saved to ./lidc_patches_all\LIDC-IDRI-0115\patches_metadata.csv, total patches: 5
Loading dicom files ... This may take a moment.


Processing patients: 100%|██████████| 4/4 [00:08<00:00,  2.08s/it]

Metadata saved to ./lidc_patches_all\LIDC-IDRI-0054\patches_metadata.csv, total patches: 5
Batch processing complete!
Output directory: ./lidc_patches_all
Total patches extracted: 29
Patients processed: 4
